In [179]:
import nltk
import pandas as pd
nltk.download('stopwords',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('wordnet',quiet=True)
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string
pd.options.mode.chained_assignment = None

In [180]:
df = pd.read_csv('../data/Twitter_Data.csv')
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [181]:
df=df.dropna()

In [182]:
#remove punctuation
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

In [183]:
df['tokenized_text'] = df.apply(lambda row: nltk.word_tokenize(row['clean_text']), axis=1)

In [184]:
stop_words=set(stopwords.words("english"))
def sw(list_words):
    filtered_sent=[]
    for w in list_words: 
        if w not in stop_words:
            filtered_sent.append(w)
    return filtered_sent
df['tokenized_text_stopwords'] = df.apply(lambda row: sw(row['tokenized_text']), axis=1)

In [185]:
# import these modules
lemmatizer = WordNetLemmatizer()
def lm(list_stem):
    lm_words=[]
    for w in list_stem:
        lm_words.append(lemmatizer.lemmatize(w,pos ="a"))
    return lm_words
# a denotes adjective in "pos"
df['tokenized_text_stopwords_lemmatized'] = df.apply(lambda row: lm(row['tokenized_text_stopwords']), axis=1)

In [186]:
#drop all unused columns
df2 = df[['tokenized_text_stopwords_lemmatized','category']]
#change data type from list to str
df2['tokenized_text_stopwords_lemmatized']=df2['tokenized_text_stopwords_lemmatized'].apply(lambda x: " ".join(x) )
df2

,tokenized_text_stopwords_lemmatized,category
0,modi promised “ minimum government maximum gov...,-1.0
1,talk nonsense continue drama vote modi,0.0
2,say vote modi welcome bjp told rahul main camp...,1.0
3,asking supporters prefix chowkidar names modi ...,1.0
4,answer among powerful world leader today trump...,1.0
...,...,...
162975,456 crores paid neerav modi recovered congress...,-1.0
162976,dear rss terrorist payal gawar modi killing 10...,-1.0
162977,cover interaction forum left,0.0
162978,big project came india modi dream project happ...,0.0


In [187]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df2['tokenized_text_stopwords_lemmatized'])
text_counts

<162969x105968 sparse matrix of type '<class 'numpy.int64'>'
	with 2007444 stored elements in Compressed Sparse Row format>

In [188]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, df2['category'], test_size=0.2, random_state=1)

In [189]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
model = MultinomialNB()
print(type(model))
# Model Generation Using Multinomial Naive Bayes
clf = model.fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

<class 'sklearn.naive_bayes.MultinomialNB'>
MultinomialNB Accuracy: 0.7006197459655151
